# Rolf Fourier functions in python
### It is now working in Python
greektype=1 Price,
greektype=2 delta,
greektype=3 Gamma,
greektype=4 Vega, 
Only the last function is not translated as we probably does not need it

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq as root
import scipy.integrate as integrate

In [2]:
def BSFourier(spot,timetoexp,strike,r,divyield,sigma,greek=1):
    X=np.log(spot/strike)+(r-divyield)*timetoexp
    def integrand(k):
        return (np.exp( (-1j*k+0.5)*X- 0.5*(k**2+0.25)*sigma**2*timetoexp)/(k**2+0.25)).real
    dummy=integrate.quad(integrand,-np.inf,np.inf)[0]
    return np.exp(-divyield*timetoexp)*spot-strike*np.exp(-r*timetoexp)*dummy/(2*np.pi)

In [3]:
def HestonFourier(spot,timetoexp,strike,r,divyield,V,theta,kappa,epsilon,rho,greek=1):
    X = np.log(spot/strike)+(r-divyield)*timetoexp
    kappahat = kappa-0.5*rho*epsilon
    xiDummy = kappahat**2+0.25*epsilon**2
    
    def integrand(k):
        xi = np.sqrt(k**2*epsilon**2*(1-rho**2)+2j*k*epsilon*rho*kappahat+xiDummy)
        PsiP = -(1j*k*rho*epsilon+kappahat)+xi
        PsiM = (1j*k*rho*epsilon+kappahat)+xi
        alpha = -kappa*theta*(PsiP*timetoexp + 2*np.log((PsiM + PsiP*np.exp(-xi*timetoexp))/(2*xi)))/epsilon**2
        beta = -(1-np.exp(-xi*timetoexp))/(PsiM + PsiP*np.exp(-xi*timetoexp))
        numerator = np.exp((-1j*k+0.5)*X+alpha+(k**2+0.25)*beta*V)

        if(greek==1):
            dummy = (numerator/(k**2+0.25)).real
        if(greek==2):
            dummy = ((0.5-1j*k)*numerator/(spot*(k**2+0.25))).real
        if(greek==3):
            dummy = -(numerator/spot**2).real
        if(greek==4):
            dummy = (numerator*beta).real
    
        return dummy

    dummy=integrate.quad(integrand,-100,100)[0] #stop.on.error = FALSE skal det med på noget måde?

    if (greek==1):
        dummy = np.exp(-divyield*timetoexp)*spot-strike*np.exp(-r*timetoexp)*dummy/(2*np.pi)

    if(greek==2):
        dummy = np.exp(-divyield*timetoexp)-strike*np.exp(-r*timetoexp)*dummy/(2*np.pi)

    if(greek==3):
        dummy = -strike*np.exp(-r*timetoexp)*dummy/(2*np.pi)
  
    if(greek==4):
        dummy = -strike*np.exp(-r*timetoexp)*dummy/(2*np.pi)

    return dummy 

In [ ]:
# I dont think we need this function.
#Andreasen.Fourier <- function(spot,timetoexp,strike,Z,lambda,beta,epsilon){

#X<-log(spot/strike)

#integrand<-function(k){
#    neweps<-lambda*epsilon
#    xi<-sqrt(k^2*neweps^2+beta^2+0.25*neweps^2)
#    Psi.P<--beta+xi
#    Psi.M<-beta+xi
#    A<--beta*(Psi.P*timetoexp + 2*log((Psi.M + Psi.P*exp(-xi*timetoexp))/(2*xi)))/(epsilon^2)
#    B<-(1-exp(-xi*timetoexp))/(Psi.M + Psi.P*exp(-xi*timetoexp))
#    integrand<-Re(exp( (-1i*k+0.5)*X+A-(k^2+0.25)*B*lambda^2*Z)/(k^2+0.25))
#  } 

#  dummy<-integrate(integrand,lower=-Inf,upper=Inf)$value

# Andreasen.Fourier<-spot-strike*dummy/(2*pi)
  
#}